# 1. 서울 도로 정보 가져오기

###1. google drive mount 및 필요 라이브러리 가져오기

In [44]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

drive = './gdrive/My Drive/Colab Notebooks/Lumos_9X/Test'
!ls -al './gdrive/My Drive/Colab Notebooks/Lumos_9X/Test/data'

Mounted at /content/gdrive
total 24
drwx------ 2 root root 4096 May  4 16:21 EMD_201902
drwx------ 2 root root 4096 May  4 13:25 gangnam
drwx------ 2 root root 4096 May  4 16:21 LI_201902
drwx------ 2 root root 4096 May  4 07:45 NodeLink
drwx------ 2 root root 4096 May  5 07:10 pdstrn
drwx------ 2 root root 4096 May  6 08:39 Seoul


In [0]:
#!pip install pyshp
#!pip install pyproj

In [0]:
import shapefile #pyshp module.
import pandas as pd
import numpy as np
from pyproj import Proj, transform #pyproj module.
import networkx as nx

###2. shp file 이용해서 dataframe에 저장하기 

In [0]:
#file 읽기 (encoding 주의)
shp_path_node = './gdrive/My Drive/Colab Notebooks/Lumos_9X/Test/data/Seoul/Z_KAIS_TL_SPRD_MANAGE_11000.shp'
sf_node = shapefile.Reader(shp_path_node, encoding = 'cp949')


In [48]:

#1. 필드명 두번째 줄부터 뽑아서 리스트 형태로 저장하기
fields_node = [x[0] for x in sf_node.fields][1:]
len(fields_node) #11

#2. 데이터 
records_node = sf_node.records()

print(len(fields_node) ,fields_node )
records_node[1:5]

20 ['ALWNC_DE', 'ALWNC_RESN', 'BSI_INT', 'ENG_RN', 'MVMN_DE', 'MVMN_RESN', 'MVM_RES_CD', 'NTFC_DE', 'OPERT_DE', 'RBP_CN', 'RDS_DPN_SE', 'RDS_MAN_NO', 'REP_CN', 'RN', 'RN_CD', 'ROAD_BT', 'ROAD_LT', 'ROA_CLS_SE', 'SIG_CD', 'WDR_RD_CD']


[Record #1: ['20100419', '중랑천의 옛 이름인 한천에서 유래', '10', 'Hancheon-ro', '20100910', '광역도로 정비 및 부여사유, 부여방식 정비', '90', '20100422', '20141030220124', '석관동 42-12대', '2', 409, '장위동 311-42 천', '한천로', '3005038', 2.0, 11840.0, '3', '11290', '2'],
 Record #2: ['20100617', '경희대로에서 분기된 도로구간으로 순차적으로 일련번호 부여', '10', 'Kyungheedae-ro 1-gil', '', '광역도로 정비 및 부여사유, 부여방식 정비', '90', '20100617', '20120306143303', '회기동 16-61', '1', 5, '회기동 60-90', '경희대로1길', '4115019', 7.0, 109.0, '4', '11230', '3'],
 Record #3: ['20100617', '홍릉로에서 분기된 도로구간으로 순차적으로 일련번호 부여', '10', 'Hongneung-ro 1ga-gil', '', '광역도로 정비 및 부여사유, 부여방식 정비', '90', '20100617', '20120306165329', '제기동 632-20', '1', 2942, '제기동 575-2', '홍릉로1가길', '4115654', 8.0, 31.0, '4', '11230', '3'],
 Record #4: ['20100514', '장위로에서 분기되는 도로구간으로 해당 일련번호 부여', '10', 'Jangwi-ro 40-gil', '20130528', '광역도로 정비 및 부여사유, 부여방식 정비', '90', '20100610', '20130528203251', '장위동 74-5대', '1', 1560, '장위동 66-294대', '장위로40길', '4121578', 5.0, 12.0, '4', '11290', '3']]

In [57]:
#4. dataframe으로 저장
##4_1. 노드 데이터
node_dataframe = pd.DataFrame(columns = fields_node, data = records_node )

print(node_dataframe.shape)
node_dataframe.head(5)

(68946, 20)


,ALWNC_DE,ALWNC_RESN,BSI_INT,ENG_RN,MVMN_DE,MVMN_RESN,MVM_RES_CD,NTFC_DE,OPERT_DE,RBP_CN,RDS_DPN_SE,RDS_MAN_NO,REP_CN,RN,RN_CD,ROAD_BT,ROAD_LT,ROA_CLS_SE,SIG_CD,WDR_RD_CD
0,20100605,용마공원로에서 분기되는 도로구간으로 일련번호부여,8,Yongmagongwon-ro 4-gil,20181213,도로구간변경,12,20100610,20181213150729,망우동 408-1,1,1518,망우동 412-49,용마공원로4길,4118463,2.0,21.0,4,11260,3
1,20100419,중랑천의 옛 이름인 한천에서 유래,10,Hancheon-ro,20100910,"광역도로 정비 및 부여사유, 부여방식 정비",90,20100422,20141030220124,석관동 42-12대,2,409,장위동 311-42 천,한천로,3005038,2.0,11840.0,3,11290,2
2,20100617,경희대로에서 분기된 도로구간으로 순차적으로 일련번호 부여,10,Kyungheedae-ro 1-gil,,"광역도로 정비 및 부여사유, 부여방식 정비",90,20100617,20120306143303,회기동 16-61,1,5,회기동 60-90,경희대로1길,4115019,7.0,109.0,4,11230,3
3,20100617,홍릉로에서 분기된 도로구간으로 순차적으로 일련번호 부여,10,Hongneung-ro 1ga-gil,,"광역도로 정비 및 부여사유, 부여방식 정비",90,20100617,20120306165329,제기동 632-20,1,2942,제기동 575-2,홍릉로1가길,4115654,8.0,31.0,4,11230,3
4,20100514,장위로에서 분기되는 도로구간으로 해당 일련번호 부여,10,Jangwi-ro 40-gil,20130528,"광역도로 정비 및 부여사유, 부여방식 정비",90,20100610,20130528203251,장위동 74-5대,1,1560,장위동 66-294대,장위로40길,4121578,5.0,12.0,4,11290,3


###3. 원하는 지역의 정보만 가져오기

Lumos test에서는 강남구 양재동 일대를 대상으로 한다.   
행자부코드 : 11680  (강남구)
권역코드     : 122 (서울특별시)

http://nodelink.its.go.kr/intro/intro06_05.aspx  << 에서 확인

In [67]:
#원하는 길만 뽑아보기
df_node_ex = node_dataframe[node_dataframe.RN.str.contains('논현로8길') == True].sort_values(["RDS_MAN_NO"])
print(df_node_ex.shape, "len : ", len(df_node_ex))
df_node_ex

#시군구코드,도로구간일련번호로 정렬 
data = node_dataframe[['RBP_CN','REP_CN', 'SIG_CD', 'RDS_MAN_NO']].sort_values(["SIG_CD","RDS_MAN_NO"])
data

(10, 20) len :  10


,RBP_CN,REP_CN,SIG_CD,RDS_MAN_NO
68245,명륜3가 1-352,명륜3가 53-21,11110,1
68250,명륜3가 1-1198,명륜3가 1-352,11110,2
68251,명륜1가 5-27,명륜1가 7-101,11110,3
27235,혜화동 142-1,명륜2가 181-4,11110,4
21098,묘동 20-5,묘동 20-4,11110,5
247,돈의동 1-1,묘동 164-1,11110,6
183,종로6가 238-19,충신동 208-4,11110,7
17520,창신동 570-4,창신동 660-123,11110,8
68213,창신동 570-3,창신동 587-1,11110,9
68241,창신동 700-10,창신동 390-9,11110,10


###6. csv로 저장

In [0]:
df_node.to_csv('./gdrive/My Drive/Colab Notebooks/Lumos_9X/Test/nodes.csv', encoding = 'utf-8' ) # node list
df_link.to_csv('./gdrive/My Drive/Colab Notebooks/Lumos_9X/Test/link.csv', encoding = 'utf-8') # edge(=link) list